In [157]:
from botasaurus.anti_detect_driver import AntiDetectDriver
from botasaurus import browser
from bs4 import BeautifulSoup
from botasaurus.utils import write_html, write_file
from math import ceil
import pandas as pd

addy_of_interest = "0x089258ed79f140d73638e1bb59ea9599603f3222"

url = f"https://dym.fyi/address/{addy_of_interest}"

def get_tx_links(driver: AntiDetectDriver):
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    driver.sleep(1)
    table = soup.find_all("table", {"class": "table"})
    all_href = table[0].find_all("a")
    step = 3
    links = []

    for i in range(0, len(all_href), step):
        # print the text of the href
        link = f"https://dym.fyi/tx/{all_href[i].text}']"
        links.append(link)

    return links



def get_url(url, driver: AntiDetectDriver):
    driver.get(url)
    driver.sleep(2)
    driver.refresh()
    driver.sleep(2)


@browser(block_images=True, reuse_driver=True, window_size=(800, 600))
def scrape_links_task(driver: AntiDetectDriver, data):
    links = []
    get_url(url=url, driver=driver)


    #getting total tx number for the address
    tx_num = driver.get_element_or_none("/html/body/main/main/div[3]/div[1]/div/div[1]")
    tx_num = tx_num.text
    tx_num = tx_num.split(" ")[1]
    tx_num = int(tx_num)
    print(tx_num)
    page_num = 0

    #Extracting the Tx links
    for i in range(ceil(tx_num/25)):
        links.extend(get_tx_links(driver))
        driver.click("div.col-sm-6:nth-child(2) > nav:nth-child(1) > ul:nth-child(1) > li:nth-child(4) > span:nth-child(1)")
        driver.sleep(2)



    links = [link.strip("']") for link in links]

    df = pd.DataFrame(links)
    df.to_csv("links.csv")


    

    return {
        "Total Links": len(links),
        "Links": links
    }








scraped_links = scrape_links_task()

Sleeping for 2 seconds...
Sleeping for 2 seconds...
69
Sleeping for 1 seconds...
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Sleeping for 2 seconds...
Written
     output/scrape_links_task.json


In [165]:
scraped_links["Links"]

['https://dym.fyi/tx/26C38437DC7ACA99068D11A0BB127203FD9AA5A22DD7100660A7D4731D3C9A6C',
 'https://dym.fyi/tx/5E5EED6E813BEB26F728C0B8BC1CF22B9E73625C3E4AA59F30347E07D673623A',
 'https://dym.fyi/tx/E0994BC7EC305345A410DFC5489FB23035FEC8A92D99A83254944F3966E79E17',
 'https://dym.fyi/tx/F33C2B227B0F0A5B8F8AE069C4B30F4BA9A20EA57711F8CCDA2325D592C0636A',
 'https://dym.fyi/tx/FCBE0428ED409D1CE875E5B638E0CB8A33C73B37393A5B0DB72BCDD017C54667',
 'https://dym.fyi/tx/9C77F05AC0FD8FCFE8CA3DAB130FDD948D890A627DD9B72D5A978BACE664C73E',
 'https://dym.fyi/tx/22740E779D30546A61EC91DD341FD73C6617C6D0F7DD86D0407D9B246BA36EDA',
 'https://dym.fyi/tx/A580AAAC89F2476E2FA46053FF1A443AB0B3309B4313BE93814FD3A81167D48B',
 'https://dym.fyi/tx/713040B7A922EB76E5A8EA9B638A3964E436E2993D9AB4DD2B8F30DE14E2A47C',
 'https://dym.fyi/tx/910DD57BF49F7E7DFA7C8B0FC555C2AC37232C088B6ED377CA1069F1537399C7',
 'https://dym.fyi/tx/D8461AB59A82F9BDFD627DCC87A5125112C04AB0A0AF2C82DDB86EBB67499FB6',
 'https://dym.fyi/tx/A81A4D8477C

In [166]:
@browser(block_images=True, reuse_driver=True, window_size=(800, 600))
def scrape_address_task(driver: AntiDetectDriver, data):
    addresses = []
    links = pd.read_csv("links.csv")
    links = links.values.tolist()
    for link in links:
        driver.get(link[1])
        driver.sleep(2)
        driver.click(".row-msg-summary")
        driver.sleep(1)
        #getting the address of the tx number
        print(f"Getting address number {link[0]} of {len(links)}")
        address = driver.get_element_or_none("/html/body/main/main/div[2]/div/div/div[1]/div/table/tbody/tr[2]/td/div/div/div[1]/div[1]/div[2]/div[1]/div[2]/a")
        address = address.text
        addresses.append(address)


    return {
        "Total Addresses": len(addresses),
        "Addresses": addresses
    }



address_list = scrape_address_task()


Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 0 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 1 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 2 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 3 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 4 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 5 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 6 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 7 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 8 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 9 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 10 of 69
Sleeping for 2 seconds...
Sleeping for 1 seconds...
Getting address number 11 of 69
Sl

In [168]:

df = pd.DataFrame(address_list)
df.to_csv("addresses.csv", index=False)